## Web Scraping

Go to Yelp, choose any city, web scrape some basic information for around 20 restaurants(two pages), including:
1. number of reviews, 
2. star rating,
3. phone number, 
4. address(location)
5. at least four amenities, and transform them into dictionary. For example, {'Dairy-Free Options': 'Yes'...}

In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [136]:
##define a globad header
headers={"accept": "*/*",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9",
"cache-control": "no-cache, must-revalidate",
"content-type": "text/javascript; charset=UTF-8"}

In [ ]:
lis1=[1,2,3]
lis2=[2]
lis1.append(lis2)--lis1=[1,2,3,[2]]
lis1.extend(lis2)--lis1=[1,2,3,2]

In [137]:
def get_urls(base_url,n):
    ##get page url
    extracted_url=[]
    for i in range(n):
        params ={
            'find_loc':'Santa Cruz',
            'ns': 1,
            'start': i*10
        }
        response = requests.get(base_url, params=params, headers=headers)
        soup = BeautifulSoup(response.text,"lxml")
        
        ##find all urls in this page
        urls=soup.find_all('a',attrs={'class':'lemon--a__373c0__IEZFH link__373c0__1G70M link-color--inherit__373c0__3dzpk link-size--inherit__373c0__1VFlE'})
        ##extract the valid ones for restuarant
        extracted_url.extend([url['href'] for url in urls if ('biz' in url['href']) and (len(url['href'])<100)])
    ##get the complete urls    
    valid_url=['https://www.yelp.com'+u for u in extracted_url]
        
    return valid_url


In [60]:
##get urls for two pages
##check whether its valid
get_urls('https://www.yelp.com/search',2)

['https://www.yelp.com/biz/mariannes-ice-cream-santa-cruz',
 'https://www.yelp.com/biz/the-penny-ice-creamery-santa-cruz',
 'https://www.yelp.com/biz/lillians-italian-kitchen-santa-cruz-2',
 'https://www.yelp.com/biz/cafe-brasil-santa-cruz',
 'https://www.yelp.com/biz/laili-restaurant-santa-cruz',
 'https://www.yelp.com/biz/lindas-seabreeze-cafe-santa-cruz',
 'https://www.yelp.com/biz/harbor-cafe-santa-cruz',
 'https://www.yelp.com/biz/verve-coffee-roasters-santa-cruz-3',
 'https://www.yelp.com/biz/hulas-island-grill-and-tiki-room-santa-cruz',
 'https://www.yelp.com/biz/walnut-avenue-caf%C3%A9-santa-cruz-2',
 'https://www.yelp.com/biz/pacific-cookie-company-santa-cruz-5',
 'https://www.yelp.com/biz/taqueria-los-pericos-santa-cruz',
 'https://www.yelp.com/biz/the-buttery-santa-cruz',
 'https://www.yelp.com/biz/zacharys-restaurant-santa-cruz',
 'https://www.yelp.com/biz/pono-hawaiian-grill-santa-cruz',
 'https://www.yelp.com/biz/mission-hill-creamery-santa-cruz-4',
 'https://www.yelp.com

In [138]:
##stars and reviews are in the same divs
def get_stars_reviews(soup):
    ###get stars_reviews:
    info = soup.find('div', attrs={'class': 'lemon--div__373c0__1mboc arrange__373c0__2C9bH gutter-1-5__373c0__2vL-3 vertical-align-middle__373c0__1SDTo margin-b1__373c0__1khoT border-color--default__373c0__3-ifU'})   
    divs = info.find_all('div')
    star = divs[1]
    reviews = divs[2]  
    return (float(star['aria-label'].split()[0]),int(reviews.text.split()[0]))

In [68]:
get_stars_reviews('https://www.yelp.com/biz/mariannes-ice-cream-santa-cruz')

(4.5, 1816)

In [150]:
#if information is in <p> </p>, we can always get it from .text
import re
def get_phone(soup):
    phone = soup.find_all('p', attrs={'class': 'lemon--p__373c0__3Qnnj text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa-'})
    ##get all texts
    lis = [p.text for p in phone]
    ##extract digits from all p
    regex=re.compile(r"\d+")
    number = [''.join(regex.findall(l)) for l in lis]
    valid = [n for n in number if n != '']
    ##get the valid phone number
    if len(valid)>0:
        return [v for v in valid if len(v)==10][0]
    else:
        return 'No Info'

In [151]:
len('8314234545')

10

In [152]:
get_phone(soup)

'8314234545'

In [164]:
def get_address(soup):
    add = soup.find_all('address',attrs={'class':'lemon--address__373c0__2sPac'})
    format_add = ' '.join([a.text for a in add])
    return format_add

In [1]:
add = ['101B Cooper St','Santa Cruz, CA 95060']
' '.join(add)

'101B Cooper St Santa Cruz, CA 95060'

In [165]:
get_address(soup)

'101B Cooper StSanta Cruz, CA 95060'

In [157]:
def get_amen(soup):
    ##get to the right division
    div = soup.find('div',attrs={'class':'lemon--div__373c0__1mboc arrange__373c0__2C9bH gutter-2__373c0__1DiLQ layout-wrap__373c0__1as1X layout-2-units__373c0__38itL border-color--default__373c0__3-ifU'})
    amen_text = div.find_all('span',attrs={'class':'lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa-'})
    amen_value = div.find_all('span',attrs={"class":"lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-weight--bold__373c0__1elNz"})
    ##combine text and values
    amen = {t.text:v.text.replace('\xa0','') for t,v in zip(amen_text,amen_value)}
    return amen   
    

In [135]:
response = requests.get('https://www.yelp.com/biz/laili-restaurant-santa-cruz',headers=headers)
soup = BeautifulSoup(response.text,"lxml")
lis = soup.find_all('div',attrs={'class':'lemon--div__373c0__1mboc arrange__373c0__2C9bH gutter-2__373c0__1DiLQ layout-wrap__373c0__1as1X layout-2-units__373c0__38itL border-color--default__373c0__3-ifU'})
amen_text = lis[0].find_all('span',attrs={'class':'lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa-'})
amen_value = lis[0].find_all('span',attrs={"class":"lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-weight--bold__373c0__1elNz"})
{t.text:v.text.replace('\xa0','') for t,v in zip(amen_text,amen_value)}

#[l for l in lis if '<span' in l]

{'Vegetarian Options': 'Yes',
 'High Chairs': 'Yes',
 'Vegan Options': 'Yes',
 'Kids Menu': 'Yes'}

In [158]:
##lastly, get the name of the restaurant
def get_name(soup):
    return soup.find('h1', attrs = {'class':'lemon--h1__373c0__2ZHSL heading--h1__373c0__dvYgw undefined heading--inline__373c0__10ozy'}).text


In [166]:
##grab all infomation together
all_names = ['Restaurant Name','Star Rating','Number of Reviews','Phone','Address','Amentities']
def all_info(url):
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text,"lxml")
    ##grab all infos
    name = get_name(soup)
    star = get_stars_reviews(soup)[0]
    review = get_stars_reviews(soup)[1]
    phone = get_phone(soup)
    address = get_address(soup)
    amen = get_amen(soup)
    info = [name,star,review,phone,address,amen]
    return dict(zip(all_names,info))

In [167]:
all_info('https://www.yelp.com/biz/laili-restaurant-santa-cruz')

{'Restaurant Name': 'Laili Restaurant',
 'Star Rating': 4.5,
 'Number of Revies': 1189,
 'Phone': '8314234545',
 'Address': '101B Cooper StSanta Cruz, CA 95060',
 'Amentities': {'Vegetarian Options': 'Yes',
  'High Chairs': 'Yes',
  'Vegan Options': 'Yes',
  'Kids Menu': 'Yes'}}

In [ ]:
##this should get the results directly, but we need to be a little more cautious
[all_info(url) for url in get_urls('https://www.yelp.com/search',2)]

In [168]:
results=[]
for url in get_urls('https://www.yelp.com/search',2):
    ##incase some page wouldnt work
    try:
        scraped = all_info(url)
    except:
        print('blocked')
    ##adding time sleep will decrease the chance of us being blocked,
    ##but increase the operation time
    time.sleep(1)
    results.append(scraped)

In [169]:
##mae a lis of dictionary
results

[{'Restaurant Name': "Marianne's Ice Cream",
  'Star Rating': 4.5,
  'Number of Revies': 1816,
  'Phone': '8314581447',
  'Address': '1020 Ocean StSanta Cruz, CA 95060',
  'Amentities': {'Dairy-Free Options': 'Yes',
   'Vegetarian Options': 'Yes',
   'Offers Delivery': 'Yes',
   'Offers Takeout': 'Yes'}},
 {'Restaurant Name': 'The Penny Ice Creamery',
  'Star Rating': 4.5,
  'Number of Revies': 1724,
  'Phone': '8312042523',
  'Address': '913 Cedar StSanta Cruz, CA 95060',
  'Amentities': {'Dairy-Free Options': 'Yes',
   'Vegan Options': 'Yes',
   'Offers Delivery': 'Yes',
   'Offers Takeout': 'Yes'}},
 {'Restaurant Name': "Lillian's Italian Kitchen",
  'Star Rating': 4.5,
  'Number of Revies': 2065,
  'Phone': '8314252288',
  'Address': '1148 Soquel AveSanta Cruz, CA 95062',
  'Amentities': {'Vegetarian Options': 'Yes',
   'High Chairs': 'Yes',
   'Kids Activities Nearby': 'Yes',
   'Happy Hour Specials': 'Yes'}},
 {'Restaurant Name': 'Cafe Brasil',
  'Star Rating': 4.0,
  'Number of 

In [170]:
##
import pandas as pd
pd.DataFrame(results)

,Restaurant Name,Star Rating,Number of Revies,Phone,Address,Amentities
0,Marianne's Ice Cream,4.5,1816,8314581447,"1020 Ocean StSanta Cruz, CA 95060","{'Dairy-Free Options': 'Yes', 'Vegetarian Opti..."
1,The Penny Ice Creamery,4.5,1724,8312042523,"913 Cedar StSanta Cruz, CA 95060","{'Dairy-Free Options': 'Yes', 'Vegan Options':..."
2,Lillian's Italian Kitchen,4.5,2065,8314252288,"1148 Soquel AveSanta Cruz, CA 95062","{'Vegetarian Options': 'Yes', 'High Chairs': '..."
3,Cafe Brasil,4.0,1798,8314291855,"1410 Mission StSanta Cruz, CA 95060","{'Vegetarian Options': 'Yes', 'High Chairs': '..."
4,Laili Restaurant,4.5,1189,8314234545,"101B Cooper StSanta Cruz, CA 95060","{'Vegetarian Options': 'Yes', 'High Chairs': '..."
5,Linda's Seabreeze Cafe,4.5,998,8314279713,"542 Seabright AveSanta Cruz, CA 95062","{'Kids Activities Nearby': 'Yes', 'Vegan Optio..."
6,Harbor Cafe,4.5,1188,8314754948,"535 7th AveSanta Cruz, CA 95062","{'Vegetarian Options': 'Yes', 'Vegan Options':..."
7,Verve Coffee Roasters,4.0,973,8316007784,"1540 Pacific AveSanta Cruz, CA 95062","{'Offers Delivery': 'No', 'Offers Takeout': 'Y..."
8,Hula's Island Grill and Tiki Room,4.0,1718,8314264852,"221 Cathcart StSanta Cruz, CA 95060","{'Vegetarian Options': 'Yes', 'High Chairs': '..."
9,Walnut Avenue Café,4.0,1036,8314572307,"106 Walnut AveSanta Cruz, CA 95060","{'Changing Tables': 'Yes', 'High Chairs': 'Yes..."
